In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
sys.path.append('../Shared')
import processing, Model, Visualization, modules

def get_label_from_filename(fname):
    for cls, label in class_to_label.items():
        if cls in fname:
            return label
    return None  # if no class found

classes = ['STC', 'WAK', 'STDUP', 'SITDN', 'UPS', 'DNS', 'KLFT', 'TPTO', 'LLF', 'LLB', 'LLS', 'KLCL', 'HS', 'TO', 'LUGF', 'LUGB'] #16 classes

Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

base_path = 'D:/Data/Gait-EMG/SIAT/SIAT_LLMD20230404/'
Sub_lst_data = [f"Sub{i:02d}"+"/Data/" for i in range(1, 41)]
Sub_lst_label = [f"Sub{i:02d}"+"/Labels/" for i in range(1, 41)]


###나중에 여기부터 for (Subject 단위) - 지금부터 1 subject 시작
ACC_lst = []
for SUB, LAB in zip(Sub_lst_data[34:], Sub_lst_label[34:]):
    print(f"Sub: {SUB} Start\n==================================\n==================================\n")
    data_lst = os.listdir(base_path + SUB)
    label_lst = os.listdir(base_path + LAB)

    class_to_label = {cls: i for i, cls in enumerate(classes)}

    dataset, labels = [], []
    for l, d in zip(label_lst, data_lst):

        data = pd.read_csv(base_path + SUB + "/" + d)
        data = data.iloc[:, 1:]   # 첫 column(time) 제거

        label = get_label_from_filename(l)

        labels.append(np.array([label] * data.shape[0]))
        dataset.append(data)
        print("\t", l, d, " ====> ", np.array([label] * data.shape[0]).shape, data.shape)

    D = np.concatenate(dataset, axis=0)
    L = np.concatenate(labels, axis=0)
    print(D.shape, L.shape)

    balanced_idx = modules.Downsample_to_balance_class(L)
    X_emg_bal, y_bal = D[balanced_idx], L[balanced_idx]
    print("\t EMG:", X_emg_bal.shape, "y:", y_bal.shape)

    window_size, step_size = 400, 20
    num_channels, num_features = X_emg_bal.shape[1], 30

    X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

    all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
    all_y = modules.y_change_to_int(all_y)
    all_X = all_X.reshape(-1, num_channels, num_features, 1)


    from sklearn.model_selection import train_test_split

    Sub_acc = []
    for feature_idx in range(num_features):
        X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
        X_feature = X_feature.reshape(-1, num_channels, 1)
        X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

        model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

        print(f"\t Feature {feature_idx} ({Features[feature_idx]}): "
            f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
        Sub_acc.append(np.max(history.history['val_accuracy'])*100)
    ACC_lst.append(Sub_acc)